### Importy

In [ ]:
import os

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml
from price_parser import Price
from tqdm.notebook import tqdm

### Choosen dataset

In [ ]:
DATASET_NUMBER = 1
DATASET_PATH = f"./data/DATA{DATASET_NUMBER}"
RESULT_PATH = f"./result/DATA{DATASET_NUMBER}"

NA_LIKE_VALUES = ["", "NULL", "null", "NaN", "NA", " "]

In [ ]:
os.makedirs(RESULT_PATH, exist_ok=True)

In [ ]:
results = {}

### Processing book data

In [ ]:
with open(f"{DATASET_PATH}/books.yaml", 'r') as f:
    books_data = yaml.safe_load(f)

df_books_raw = pd.DataFrame(books_data)
df_books = df_books_raw.copy()
df_books.columns

In [ ]:
df_books.columns = df_books.columns.str.replace(":", "")

In [ ]:
df_books = df_books.convert_dtypes()

In [ ]:
df_books["year"] = pd.to_numeric(df_books["year"], errors="coerce")
df_books["year"] = df_books["year"].astype("Int64")

In [ ]:
df_books_cleaned = df_books.copy()

In [ ]:
df_books_cleaned

### Processing orders data

In [ ]:
df_orders_raw = pd.read_parquet(f"{DATASET_PATH}/orders.parquet")
df_orders = df_orders_raw.copy()

In [ ]:
df_orders = df_orders.convert_dtypes()

In [ ]:
df_orders['shipping'] = df_orders['shipping'].replace(NA_LIKE_VALUES, pd.NA)

In [ ]:
df_orders["timestamp_regex_clean"] = (
    df_orders["timestamp"]
    .str.replace("A.M.", "AM")
    .str.replace("P.M.", "PM")
    .str.replace(r"[,;]", ' ', regex = True))

In [ ]:
df_orders["timestamp_clean"] = pd.to_datetime(df_orders["timestamp_regex_clean"], format="mixed")

In [ ]:
df_orders["date"] = pd.to_datetime(df_orders["timestamp_clean"]).dt.date
df_orders["time"] = pd.to_datetime(df_orders["timestamp_clean"]).dt.time

In [ ]:
df_orders["unit_price_clean"]= (
    df_orders["unit_price"].astype(str)
    .str.replace(r"^([0-9]+)€([0-9]+)¢$", r"\1.\2€", regex=True)
    .str.replace(r"^€([0-9]+)¢([0-9]+)$", r"\1.\2€", regex=True)
    .str.replace(r"^([0-9]+)\$([0-9]+)¢$", r"\1.\2$", regex=True)
    .str.replace(r"^\$([0-9]+)¢([0-9]+)$", r"\1.\2$", regex=True)
    .str.replace('USD', '$')
    .str.replace('EUR', '€')
)

In [ ]:
df_orders["price"] = df_orders["unit_price_clean"].apply(
    lambda p: Price.fromstring(p).amount_float if pd.notna(p) else pd.NA
)

df_orders["currency"] = df_orders["unit_price_clean"].apply(
    lambda p: Price.fromstring(p).currency if pd.notna(p) else pd.NA
)


In [ ]:
df_orders['price_in_$'] = df_orders['price'].where(
    df_orders['currency'] == '$',
    df_orders['price'] * 1.2
).round(2)

In [ ]:
df_orders['paid_price'] = df_orders['price_in_$'] * df_orders['quantity']

In [ ]:
df_orders_cleaned = pd.DataFrame(df_orders, columns=['id', 'user_id','book_id', 'quantity', 'paid_price', 'date', 'shipping'])

In [ ]:
df_orders_cleaned

### Processing users data

In [ ]:
df_users = pd.read_csv(f"{DATASET_PATH}/users.csv")

In [ ]:
df_users = df_users.convert_dtypes()

In [ ]:
df_users['address'] = df_users['address'].replace(NA_LIKE_VALUES, pd.NA)

In [ ]:
df_users["clean_phone"] = (
    df_users["phone"]
    .str.replace(r"[()\.-]", "", regex=True)
    .str.replace(r"\s+", "", regex=True)
)

In [ ]:
df_users['clean_phone'] = df_users['clean_phone'].replace(NA_LIKE_VALUES, pd.NA)

In [ ]:
df_users_cleaned = pd.DataFrame( df_users, columns=['id', 'name', 'address', 'clean_phone', 'email'])

In [ ]:
df_users_cleaned

### Task1: Compute daily revenue (sum of `paid_price` grouped by `date`) and find top 5 days by revenue.

In [ ]:
daily_revenue = df_orders_cleaned.groupby('date')['paid_price'].sum().reset_index().rename(columns={'paid_price': 'daily_revenue'})

In [ ]:
top_five_revenues = daily_revenue.sort_values(by='daily_revenue', ascending=False, ignore_index=True).head(5)
top_five_revenues

#### Save result

In [ ]:
top_five_revenues.to_csv(f"{RESULT_PATH}/top_five_revenues.csv", index = False, float_format="%.2f")

### Task 2: Find how many real unique users there are. Note that user can change address or change phone or even provide alias instead of a real name; you need to reconciliate data. You may assume that only one field is changed.

In [ ]:
df_unique_users = df_users_cleaned.copy()

In [ ]:
df_unique_users['address'] = df_unique_users['address'].fillna('')

In [ ]:
df_unique_users['new_user_id'] = None

In [ ]:
for index, row in tqdm(df_unique_users.iterrows(), total=len(df_unique_users)):

    if row['new_user_id'] is None:
        df_unique_users.at[index, 'new_user_id'] = index

    for index2, row2 in df_unique_users.iterrows():
        if index2 == index:
            continue

        same_columns_count = 0
        for col in ['name', 'address', 'clean_phone', 'email']:
            if row[col] == row2[col]:
                same_columns_count += 1

          
        if same_columns_count >= 3:
            df_unique_users.at[index2, 'new_user_id'] = df_unique_users.at[index, 'new_user_id']

df_unique_users

In [ ]:
df_unique_users['new_user_id'].value_counts()

In [ ]:
real_unique_users = df_unique_users['new_user_id'].nunique()
real_unique_users

#### Save result

In [ ]:
results["real_unique_users"] = real_unique_users

### Task 3: Find how how many unique sets of authors there are. For example, if `John` and `Paul` wrote a book together and wrote several books separately, it means that there are 3 different sets.


In [ ]:
df_unique_authors = df_books_cleaned.copy()

In [ ]:
df_unique_authors['authors_list'] = df_unique_authors['author'].str.split(',')

In [ ]:
df_unique_authors["author_set"] = df_unique_authors["authors_list"].apply(frozenset)

In [ ]:
df_unique_authors

In [ ]:
unique_sets = df_unique_authors["author_set"].nunique()
unique_sets

#### Save result

In [ ]:
results['unique sets of authors'] = unique_sets

 ### Task 4: Find the most popular (by sold book count) author (or author set).

In [ ]:
df_orders_cleaned

In [ ]:
df_books_cleaned

In [ ]:
df_merged = df_orders_cleaned.merge(df_books_cleaned, left_on='book_id', right_on='id')

In [ ]:
the_most_popular_author = df_merged.groupby('author')['quantity'].count().reset_index().rename(columns={'quantity': 'sold_book_count'}).sort_values(by='sold_book_count', ascending=False).head(1)
the_most_popular_author

#### Save result

In [ ]:
the_most_popular_author.to_csv(f"{RESULT_PATH}/the_most_popular_author.csv", index = False)

### Task 5 Identify the top customer by total spending (list all `user_id` values for the possible different addresses, phones, e-mails, or aliases).

In [ ]:
df_orders_cleaned

In [ ]:
df_unique_users

In [ ]:
df_merged_top_customer = df_orders_cleaned.merge(df_unique_users, how ='right', left_on='user_id', right_on='id')

In [ ]:
top_customer = df_merged_top_customer.groupby('new_user_id')['paid_price'].sum().reset_index().sort_values(by='paid_price', ascending = False).rename(columns={'paid_price': 'total spending'})
top_customer

In [ ]:
top_customer_data = df_unique_users[df_unique_users['new_user_id'] == top_customer.reset_index().loc[0, 'new_user_id']]

In [ ]:
top_customer_data

In [ ]:
ids = top_customer_data['id'].tolist()
ids

#### Save result

In [ ]:
top_customer_data.to_csv(f"{RESULT_PATH}/top_customer_data.csv", index = False)

### Task 6: Plot a simple line chart of daily revenue using matplotlib.

In [ ]:
daily_revenue

In [ ]:

fig, ax = plt.subplots(figsize=(20, 5))
ax.plot(daily_revenue['date'], daily_revenue['daily_revenue'])

ax.set_title("Daily revenue")
ax.set_xlabel("Date")
ax.set_ylabel("Revenue [$]")

fig.autofmt_xdate()

ax.grid(True)
ax.xaxis.set_major_locator(mdates.MonthLocator())

#### Save result

In [ ]:
fig.savefig(f"{RESULT_PATH}/daily_revenue.png", dpi=150)
plt.close(fig)

### Results to csv

In [ ]:
results.items()

In [ ]:
df_results = pd.DataFrame(results.items(), columns=['information', 'number'])

In [ ]:
df_results

In [ ]:
df_results.to_csv(f"{RESULT_PATH}/unique_results.csv", index = False)